In [ ]:
import os
import json
from dotenv import dotenv_values
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FileSearchTool, MessageAttachment, FilePurpose
from azure.identity import DefaultAzureCredential

In [ ]:
env = dotenv_values(".env")

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=env["FOUNDRY_CONN"]
)

ny_file = project_client.agents.upload_file_and_poll(file_path='../../city_info/new_york.txt', purpose=FilePurpose.AGENTS)
london_file = project_client.agents.upload_file_and_poll(file_path='../../city_info/london.txt', purpose=FilePurpose.AGENTS)
tokyo_file = project_client.agents.upload_file_and_poll(file_path='../../city_info/tokyo.txt', purpose=FilePurpose.AGENTS)

vector_store = project_client.agents.create_vector_store_and_poll(file_ids=[ny_file.id, london_file.id, tokyo_file.id], name="city_info_vector_store")


In [ ]:
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="tourist-agent",
    instructions="You are a tourist info bot. Use the provided tools and resources to answer questions.",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
)

In [ ]:
thread = project_client.agents.create_thread()

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Suggest an interesting thing to do in London today.",
)


In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")